### Experiments for Reconstruction of networks using (p)FBA 

In [ ]:
# Ensure necessary packages are installed
!pip install cobra efmtool numpy pandas scipy

Definition of the reduced DFKI model without information about the reversibility of reactions.

In [3]:
import cobra
from cobra import Model, Reaction, Metabolite

# Modell erstellen
model = Model('Toy_Model')

# Metaboliten erstellen
A_EX = Metabolite('A_EX', compartment='e')
A = Metabolite('A', compartment='c')
B = Metabolite('B', compartment='c')
D = Metabolite('D', compartment='c')
E = Metabolite('E', compartment='c')
E_EX = Metabolite('E_EX', compartment='e')
F = Metabolite('F', compartment='c')
G = Metabolite('G', compartment='c')
H = Metabolite('H', compartment='c')
I = Metabolite('I', compartment='c')
J = Metabolite('J', compartment='c')
K = Metabolite('K', compartment='c')
L = Metabolite('L', compartment='c')
L_EX = Metabolite('L_EX', compartment='e')
M = Metabolite('M', compartment='c')
M_EX = Metabolite('M_EX', compartment='e')
N = Metabolite('N', compartment='c')
N_EX = Metabolite('N_EX', compartment='e')
O = Metabolite('O', compartment='c')
P = Metabolite('P', compartment='c')
P_EX = Metabolite('P_EX', compartment='e')

# Reaktionen erstellen
a_in = Reaction('A_in')
a_in.name = 'A_EX to A'
a_in.add_metabolites({A_EX: -1.0, A: 1.0})

reaction1 = Reaction('R1')
reaction1.name = 'A and G to B'
reaction1.add_metabolites({A: -1.0, G: -1.0, B: 1.0})

reaction2 = Reaction('R2')
reaction2.name = 'B to D'
reaction2.add_metabolites({B: -1.0, D: 1.0})

reaction3 = Reaction('R3')
reaction3.name = 'D to E and F'
reaction3.add_metabolites({D: -1.0, E: 1.0, F: 1.0})

reaction4 = Reaction('R4')
reaction4.name = 'B to I'
reaction4.add_metabolites({B: -1.0, I: 1.0})

reaction5 = Reaction('R5')
reaction5.name = 'I to M and J'
reaction5.add_metabolites({I: -1.0, M: 1.0, J: 1.0})

reaction6 = Reaction('R6')
reaction6.name = 'O to P and J'
reaction6.add_metabolites({O: -1.0, J: 1.0, P: 1.0})

reaction7 = Reaction('R7')
reaction7.name = 'H to K and L'
reaction7.add_metabolites({H: -1.0, K: 1.0, L: 1.0})

reaction8 = Reaction('R8')
reaction8.name = 'K and M to N'
reaction8.add_metabolites({K: -1.0, M: -1.0, N: 1.0})

reaction9 = Reaction('R9')
reaction9.name = 'A and J to I'
reaction9.add_metabolites({A: -1.0, J: -1.0, I: 1.0})

reaction10 = Reaction('R10')
reaction10.name = 'F to G'
reaction10.add_metabolites({F: -1.0, G: 1.0})

reaction11 = Reaction('R11')
reaction11.name = 'E to H'
reaction11.add_metabolites({E: -1.0, H: 1.0})

reaction12 = Reaction('R12')
reaction12.name = 'I to P'
reaction12.add_metabolites({I: -1.0, P: 1.0})

E_EX_out = Reaction('E_EX_out')
E_EX_out.name = 'E to E_EX'
E_EX_out.add_metabolites({E: -1.0, E_EX: 1.0})

L_EX_out = Reaction('L_EX_out')
L_EX_out.name = 'L to L_EX'
L_EX_out.add_metabolites({L: -1.0, L_EX: 1.0})

M_EX_out = Reaction('M_EX_out')
M_EX_out.name = 'M to M_EX'
M_EX_out.add_metabolites({M: -1.0, M_EX: 1.0})

N_EX_out = Reaction('N_EX_out')
N_EX_out.name = 'N to N_EX'
N_EX_out.add_metabolites({N: -1.0, N_EX: 1.0})

P_EX_out = Reaction('P_EX_out')
P_EX_out.name = 'P to P_EX'
P_EX_out.add_metabolites({P: -1.0, P_EX: 1.0})



# Reaktionen zum Modell hinzufügen
model.add_reactions([a_in, reaction1, reaction2, reaction3, reaction4, reaction5, 
                     reaction6, reaction7, reaction8, reaction9, reaction10,
                     reaction11, reaction12, E_EX_out, L_EX_out, M_EX_out, N_EX_out, P_EX_out])

# Setze lower bounds zu -1.000 da die Reaktionen möglicherweise reversibel sein könnten.
for reaction in model.reactions:
    if not any(met.id.endswith('_EX') for met in reaction.metabolites):
        reaction.lower_bound = -1000.0

# Exchange Reaktionen für extracelluläre Metaboliten
model.add_boundary(A_EX, type="exchange")
model.add_boundary(E_EX, type="exchange")
model.add_boundary(L_EX, type="exchange")
model.add_boundary(M_EX, type="exchange")
model.add_boundary(N_EX, type="exchange")
model.add_boundary(P_EX, type="exchange")

# Modellinformationen ausgeben
print("Reaktionen im Modell:", len(model.reactions))
print("Metaboliten im Modell:", len(model.metabolites))
print("Gene im Modell:", len(model.genes))


Reaktionen im Modell: 24
Metaboliten im Modell: 21
Gene im Modell: 0


In [4]:
import random
from cobra import Reaction

# Erstellen von Reaktionen mit 1-2 Substrat(en) und 1-2 Produkt(en)
def generate_random_reactions(model, num_reactions=2000):
    # Filter out metabolites that end with '_EX'
    metabolites = [met for met in model.metabolites if not met.compartment == 'e']
    random_reactions = []

    for i in range(num_reactions):
        reaction_valid = False

        while not reaction_valid:
            # Zufällige Auswahl der Anzahl an Substraten und Produkten
            num_reactants = random.randint(1, 2)
            num_products = random.randint(1, 2)

            # Zufällige Auswahl der Substrate und Produkte
            reactants = random.sample(metabolites, num_reactants)
            products = random.sample(metabolites, num_products)

            # Check, dass auf beiden Seiten Metaboliten vorhanden sind
            if set(reactants).isdisjoint(set(products)):
                reaction_valid = True

                # Erstellung der Reaktion
                reaction = Reaction(f'Random_Rxn_{i+1}')
                reaction.name = f'Random Reaction {i+1}'

                reaction.add_metabolites({
                    met: -1.0 for met in reactants
                })
                reaction.add_metabolites({
                    met: 1.0 for met in products
                })

                # 50% Chance, die Reaktion reversibel zu machen
                if random.random() < 0.5:
                    reaction.lower_bound = -1000.0
                else:
                    reaction.lower_bound = 0.0

                reaction.upper_bound = 1000.0

                random_reactions.append(reaction)

    return random_reactions

# Generierung von 2000 zufälligen Reaktionen + den Inversen
random_reactions_list = generate_random_reactions(model, num_reactions=2000)

# Anzahl der insgesamt generierten Reaktionen
print(f"\nTotal number of random reactions generated: {len(random_reactions_list)}")


Total number of random reactions generated: 2000


Flux values for external reactions from Elementary Flux Modes of the complete Network.

In [5]:
# Liste der experimentellen Datenpunkten mit eingefügten 0 für Produkte, die nicht erzeugt werden.
experimental_data_zeropadded = [
    {'N_EX': 0.5, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 0.0, 'M_EX': 0.0},
    {'N_EX': 0.0, 'E_EX': 1.0, 'P_EX': 0.0, 'L_EX': 0.0, 'M_EX': 0.0},
    {'N_EX': 0.0, 'E_EX': 0.0, 'P_EX': 1.0, 'L_EX': 1.0, 'M_EX': 0.0},
    {'N_EX': 0.5, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 2.0, 'M_EX': 0.0},
    {'N_EX': 0.0, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 3.0, 'M_EX': 1.0},
    {'N_EX': 0.0, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 0.0, 'M_EX': 1.0},
    {'N_EX': 0.0, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 1.5, 'M_EX': 1.0},
    {'N_EX': 0.5, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 1.25, 'M_EX': 0.0},
    {'N_EX': 0.0, 'E_EX': 0.0, 'P_EX': 1.0, 'L_EX': 0.0, 'M_EX': 0.0},
    {'N_EX': 0.5, 'E_EX': 0.0, 'P_EX': 0.0, 'L_EX': 0.5, 'M_EX': 0.0},
    {'N_EX': 0.25, 'E_EX': 0.0, 'P_EX': 0.5, 'L_EX': 0.0, 'M_EX': 0.0}
]

Method to display flux values of a model that match a certain datapoint.

In [6]:
def show_flux_distributions(model, experimental_data):
    for i, datapoint in enumerate(experimental_data):
        print(f"\nDatapoint {i+1}:")

        # Festsetzen der Flux-Werte gemäß des Datenpuktes
        with model:
            for metabolite_id, value in datapoint.items():
                reaction_id = f"{metabolite_id}_out"
                reaction = model.reactions.get_by_id(reaction_id)
                reaction.lower_bound = value
                reaction.upper_bound = value

            
            solution = model.optimize()
            
            if solution.status == 'optimal':
                if len(check_mass_balance(model, solution))==0:
                    print("Feasible solution found:")
                    print("\nFlux distribution:")
                    for reaction in model.reactions:
                        if abs(reaction.flux) > 1e-6:  # Nur Reaktionen mit Fluxes ungleich null werden angezeigt.
                            print(f"{reaction.id}: {reaction.flux:.4f}")
                else:
                    print("Mass balance does not check out")
            else:
                print("No feasible solution for this datapoint.")

Methods that check whether a certain datapoint is feasible using Flux Balance Analysis.

In [7]:
def check_mass_balance(model, solution, tolerance=0):
    imbalances = []
    fluxes = solution.fluxes.to_dict()
    
    for metabolite in model.metabolites:
        net_flux = 0.0
        for reaction in metabolite.reactions:
            reaction_id = reaction.id
            if reaction_id in fluxes:
                flux = fluxes[reaction_id]
                stoichiometry = reaction.metabolites.get(metabolite, 0)
                net_flux += flux * stoichiometry
            
        if abs(net_flux) > tolerance:
            imbalances.append((metabolite.id, net_flux))
            

    return imbalances

def is_datapoint_feasible(model, datapoint):
    with model:
         # Festsetzen der Flux-Werte gemäß des Datenpuktes
        for metabolite_id, value in datapoint.items():
            reaction_id = f"{metabolite_id}_out"
            if reaction_id in model.reactions:
                reaction = model.reactions.get_by_id(reaction_id)
                reaction.lower_bound = value
                reaction.upper_bound = value
            else:
                #print(f"Error: Reaction {reaction_id} not found in model.")
                return False

        
        solution = model.optimize()

        # Check, ob eine Lösung gefunden wurde
        if solution.status != 'optimal':
            #print(f"Error: Optimization status for datapoint is {solution.status}.")
            return False

        # Überprüfung, ob Steady-State gewahrt wurde
        imbalances = check_mass_balance(model, solution)

        
        return len(imbalances) == 0

def check_feasibility_for_all_datapoints(model, experimental_data):
    feasible_datapoints = []
    for i, datapoint in enumerate(experimental_data):
        if is_datapoint_feasible(model, datapoint):
            feasible_datapoints.append(i)
            #print(f"Datapoint {i+1} is feasible and mass balanced.")
        #else:
            #print(f"Datapoint {i+1} is not feasible or fails mass balance check.")
    return feasible_datapoints

In [8]:
print(check_feasibility_for_all_datapoints(model, experimental_data_zeropadded))

[1, 5, 9]


/Users/philipp/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [9]:
import cobra

def check_for_deadends(model):    

    # Lists for results
    upstream_no_consumption_roots = []
    downstream_no_production_roots = []

    # Initialize dictionaries for production and consumption of metabolites
    metabolite_produced = {met: False for met in model.metabolites if met.compartment == 'c'}
    metabolite_consumed = {met: False for met in model.metabolites if met.compartment == 'c'}

    # Iterate over all reactions in the model
    for reaction in model.reactions:
        if reaction.reversibility:
            #print(reaction)
            # If reversible, treat all metabolites as both reactants and products
            for met in reaction.metabolites:
                metabolite_produced[met] = True
                metabolite_consumed[met] = True
        else:
            # Metabolites produced in this reaction
            for product in reaction.products:
                #print(reaction.products)
                if product.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_produced[product] = True

            # Metabolites consumed in this reaction
            for reactant in reaction.reactants:
                
                if reactant.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_consumed[reactant] = True

    # Determine Upstream No-Consumption Roots (produced but not consumed)
    for met, produced in metabolite_produced.items():
        if produced and not metabolite_consumed[met]:
            upstream_no_consumption_roots.append(met.id)

    # Determine Downstream No-Production Roots (consumed but not produced)
    for met, consumed in metabolite_consumed.items():
        if consumed and not metabolite_produced[met]:
            downstream_no_production_roots.append(met.id)

    #print("Upstream No-Consumption Roots:", upstream_no_consumption_roots)
    #print("Downstream No-Production Roots:", downstream_no_production_roots)

    return len(upstream_no_consumption_roots) + len(downstream_no_production_roots)



Algorithm for reconstruction of a model by greedily adding reactions, that satisfy as many as possible datapoints.

In [10]:
def score_reaction(model, reaction, experimental_data):
    # Initialisiere den Score
    score = 0
    
    # Kopiere das Modell und füge die Reaktion hinzu
    with model:
        model.add_reactions([reaction])
        
        # Prüfe die Feasibility der Datenpunkte
        newly_feasible = check_feasibility_for_all_datapoints(model, experimental_data)
        num_roots = check_for_deadends(model)
        #print(num_roots)
        score = len(newly_feasible) - num_roots
        
    return score

def add_best_reaction(original_model, experimental_data, random_reactions_list, max_iterations=2000):
    # Kopiere das Modell
    model = original_model.copy()
    model.reactions.A_in.lower_bound = 1.0
    model.reactions.A_in.upper_bound = 1.0
    
    initial_feasible = check_feasibility_for_all_datapoints(model, experimental_data)
    print(f"Initial feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
    
    iteration = 0
    while len(initial_feasible) < len(experimental_data) and iteration < max_iterations:
        if not random_reactions_list:
            print("No more random reactions to add.")
            break
        
        # Scoring der Reaktionen
        reaction_scores = {reaction.id: score_reaction(model, reaction, experimental_data) for reaction in random_reactions_list}
        
        # Auswahl der besten Reaktion basierend auf dem Score
        best_reaction_id = max(reaction_scores, key=reaction_scores.get)
        best_reaction = next(r for r in random_reactions_list if r.id == best_reaction_id)
        
        print(f"Adding reaction: {best_reaction.id} with score: {reaction_scores[best_reaction_id]}")
        model.add_reactions([best_reaction])
        random_reactions_list.remove(best_reaction)
        
        # Prüfen der neuen Feasibility
        newly_feasible = check_feasibility_for_all_datapoints(model, experimental_data)
        newly_feasible_set = set(newly_feasible) - set(initial_feasible)
        
        if newly_feasible_set:
            initial_feasible = newly_feasible
            print(f"Iteration {iteration}: Added {best_reaction.id}. Now feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
            show_flux_distributions(model, experimental_data)
        else:
            # Entfernen der Reaktion, falls keine neuen Punkte feasible werden
            model.reactions.remove(best_reaction)
            print(f"Iteration {iteration}: Added {best_reaction.id}, but no new feasible data points were found.")
        
        iteration += 1

    print(f"Final feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
    if len(initial_feasible) == len(experimental_data):
        print("All data points are feasible with the added reactions!")
    else:
        print(f"Stopped after {iteration} iterations. Some data points remain infeasible.")

    return model


In [11]:
import numpy as np
random.seed(42)
np.random.seed(42)
random_reactions_list = generate_random_reactions(model, 50)

In [12]:
random.seed(42)
np.random.seed(42)

In [13]:
modified_model = add_best_reaction(model, experimental_data_zeropadded, random_reactions_list)

Initial feasible data points: 3 / 11
Adding reaction: Random_Rxn_5 with score: 4
Iteration 0: Added Random_Rxn_5. Now feasible data points: 4 / 11

Datapoint 1:
No feasible solution for this datapoint.

Datapoint 2:
Feasible solution found:

Flux distribution:
A_in: 1.0000
R1: 1.0000
R2: 1.0000
R3: 1.0000
R10: 1.0000
E_EX_out: 1.0000
EX_A_EX: -1.0000
EX_E_EX: 1.0000

Datapoint 3:
No feasible solution for this datapoint.

Datapoint 4:
No feasible solution for this datapoint.

Datapoint 5:
No feasible solution for this datapoint.

Datapoint 6:
Feasible solution found:

Flux distribution:
A_in: 1.0000
R5: 1.0000
R9: 1.0000
M_EX_out: 1.0000
EX_A_EX: -1.0000
EX_M_EX: 1.0000

Datapoint 7:
No feasible solution for this datapoint.

Datapoint 8:
No feasible solution for this datapoint.

Datapoint 9:
Feasible solution found:

Flux distribution:
A_in: 1.0000
R1: 1.0000
R2: 1.0000
R3: 1.0000
R10: 1.0000
R11: 1.0000
R12: 1.0000
P_EX_out: 1.0000
EX_A_EX: -1.0000
EX_P_EX: 1.0000
Random_Rxn_5: 1.0000


In [164]:
random.seed(17)
np.random.seed(17)
random_reactions_list = generate_random_reactions(model, 50)

In [165]:
random.seed(17)
np.random.seed(17)

In [166]:
modified_model17 = add_best_reaction(model, experimental_data_zeropadded, random_reactions_list)

Initial feasible data points: 3 / 11
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
2
1
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
Adding reaction: Random_Rxn_50 with score: 3
Iteration 0: Added Random_Rxn_50. Now feasible data points: 4 / 11

Datapoint 1:
No feasible solution for this datapoint.

Datapoint 2:
Feasible solution found:

Flux distribution:
A_in: 1.0000
R1: 1.0000
R2: 1.0000
R3: 1.0000
R10: 1.0000
E_EX_out: 1.0000
EX_A_EX: -1.0000
EX_E_EX: 1.0000

Datapoint 3:
No feasible solution for this datapoint.

Datapoint 4:
No feasible solution for this datapoint.

Datapoint 5:
No feasible solution for this datapoint.

Datapoint 6:
Feasible solution found:

Flux distribution:
A_in: 1.0000
R5: 1.0000
R9: 1.0000
M_EX_out: 1.0000
EX_A_EX: -1.0000
EX_M_EX: 1.0000

Datapoint 7:
No feasible solution for this datapoint.

Datapoint 8:
No feasible solution for this datapoint.

Datapoint 9:
No feasible solution for this datapoint.

Datapoint 10:
Feasible solution found:

Flux distribut

In [167]:
modified_model17.reactions

[<Reaction A_in at 0x28ad26750>,
 <Reaction R1 at 0x28ad26190>,
 <Reaction R2 at 0x28ad25950>,
 <Reaction R3 at 0x28ad25790>,
 <Reaction R4 at 0x28ad25290>,
 <Reaction R5 at 0x28ad25bd0>,
 <Reaction R6 at 0x28ad27890>,
 <Reaction R7 at 0x28ad25090>,
 <Reaction R8 at 0x28ad25e50>,
 <Reaction R9 at 0x28ad25490>,
 <Reaction R10 at 0x28af9ce90>,
 <Reaction R11 at 0x28af9e850>,
 <Reaction R12 at 0x28af9fe50>,
 <Reaction E_EX_out at 0x28af9da90>,
 <Reaction L_EX_out at 0x28af9c210>,
 <Reaction M_EX_out at 0x28af9e2d0>,
 <Reaction N_EX_out at 0x28af9fbd0>,
 <Reaction P_EX_out at 0x107c3cc50>,
 <Reaction EX_A_EX at 0x28d4482d0>,
 <Reaction EX_E_EX at 0x28d44bed0>,
 <Reaction EX_L_EX at 0x28d448e10>,
 <Reaction EX_M_EX at 0x28d44b2d0>,
 <Reaction EX_N_EX at 0x28d449310>,
 <Reaction EX_P_EX at 0x28d449350>,
 <Reaction Random_Rxn_50 at 0x28afb1310>,
 <Reaction Random_Rxn_47 at 0x28afb3610>,
 <Reaction Random_Rxn_34 at 0x28ca1cad0>]

## Using PFBA

In the following the same approach as above is implemented. Instead of FBA, pFBA is used

In [105]:
from cobra.flux_analysis import pfba

In [106]:
def show_flux_distributions_pfba(model, experimental_data):
    for i, datapoint in enumerate(experimental_data):
        print(f"\nDatapoint {i+1}:")

        # Temporäres Modell für jeden Datenpunkt
        with model:
            # Setze Flux-Werte gemäß des Datenpunkts
            try:
                for metabolite_id, value in datapoint.items():
                    if metabolite_id.endswith('_EX'):
                        reaction_id = f"{metabolite_id}_out"
                    if reaction_id in model.reactions:
                        reaction = model.reactions.get_by_id(reaction_id)
                        reaction.lower_bound = value
                        reaction.upper_bound = value
                    else:
                        print(f"Reaction {reaction_id} not found in the model. Skipping.")
                
                # Berechne pFBA-Lösung
                solution = pfba(model)
                
                # Überprüfe die Lösung
                if solution.status == 'optimal':
                    print("Feasible solution found.")
                    print("\nFlux distribution (non-zero fluxes):")
                    for reaction_id, flux in solution.fluxes.items():
                        if abs(flux) > 1e-6:  # Nur Reaktionen mit signifikanten Flüssen
                            print(f"{reaction_id}: {flux:.4f}")
                else:
                    print("No feasible solution for this datapoint.")

            except Exception as e:
                print(f"An error occurred while processing datapoint {i+1}: {e}")


Method to check, whether the solutions for datapoints include negative flux values for some reactions

In [107]:
def get_negative_flux_reactions_pfba(model, experimental_data):
    negative_flux_reactions = set()

    for i, datapoint in enumerate(experimental_data):
        print(f"\nProcessing Datapoint {i+1}:")

        # Temporäres Modell für jeden Datenpunkt
        with model:
            try:
                # Setze Flux-Werte gemäß des Datenpunkts
                for metabolite_id, value in datapoint.items():
                    if metabolite_id.endswith('_EX'):
                        reaction_id = f"{metabolite_id}_out"
                    if reaction_id in model.reactions:
                        reaction = model.reactions.get_by_id(reaction_id)
                        reaction.lower_bound = value
                        reaction.upper_bound = value
                    else:
                        print(f"Reaction {reaction_id} not found in the model. Skipping.")

                # Berechne pFBA-Lösung
                solution = pfba(model)

                # Überprüfe die Lösung
                if solution.status == 'optimal':
                    print("Feasible solution found.")

                    # Sammle Reaktionen mit negativem Fluss
                    for reaction_id, flux in solution.fluxes.items():
                        if flux < 0:  # Negative Flüsse
                            negative_flux_reactions.add(reaction_id)
                else:
                    print("No feasible solution for this datapoint.")

            except Exception as e:
                print(f"An error occurred while processing datapoint {i+1}: {e}")

    return list(negative_flux_reactions)

In [108]:
def check_mass_balance_pfba(model, solution, tolerance=0):
    imbalances = []
    fluxes = solution.fluxes.to_dict()
    
    for metabolite in model.metabolites:
        net_flux = 0.0
        for reaction in metabolite.reactions:
            reaction_id = reaction.id
            if reaction_id in fluxes:
                flux = fluxes[reaction_id]
                stoichiometry = reaction.metabolites.get(metabolite, 0)
                net_flux += flux * stoichiometry
            
        if abs(net_flux) > tolerance:
            imbalances.append((metabolite.id, net_flux))
            

    return imbalances

def is_datapoint_feasible_pfba(model, datapoint):
    with model:
         # Festsetzen der Flux-Werte gemäß des Datenpuktes
        for metabolite_id, value in datapoint.items():
            reaction_id = f"{metabolite_id}_out"
            if reaction_id in model.reactions:
                reaction = model.reactions.get_by_id(reaction_id)
                reaction.lower_bound = value
                reaction.upper_bound = value
            else:
                #print(f"Error: Reaction {reaction_id} not found in model.")
                return False
        try:
            # Probiere die Optimierung durchzuführen (hier pFBA)
            solution = pfba(model)
            imbalances = check_mass_balance_pfba(model, solution)
            # Prüfe, ob die Lösung optimal ist
            if solution.status == 'optimal':
                if len(imbalances) == 0:
                    return True
                else:
                    print("Mass balance check doesn't check out")
            else:
                return False
    
        except Exception as e:
            # Fehler abfangen, die während der Optimierung auftreten
            print(f"Error during optimization: {str(e)}")
            return False
        
def check_feasibility_for_all_datapoints_pfba(model, experimental_data):
    feasible_datapoints = []
    for i, datapoint in enumerate(experimental_data):
        if is_datapoint_feasible_pfba(model, datapoint):
            feasible_datapoints.append(i)
            #print(f"Datapoint {i+1} is feasible and mass balanced.")
        #else:
            #print(f"Datapoint {i+1} is not feasible or fails mass balance check.")
    return feasible_datapoints

In [109]:
def score_reaction_pfba(model, reaction, experimental_data):
    # Initialisiere den Score
    score = 0
    
    # Kopiere das Modell und füge die Reaktion hinzu
    with model:
        model.add_reactions([reaction])
        
        # Prüfe die Feasibility der Datenpunkte
        newly_feasible = check_feasibility_for_all_datapoints_pfba(model, experimental_data)
        #num_roots = check_for_deadends(model)
        score = len(newly_feasible) #- num_roots
        
    return score

def add_best_reaction_pfba(original_model, experimental_data, random_reactions_list, max_iterations=2000):
    # Kopiere das Modell
    model = original_model.copy()
    model.reactions.A_in.lower_bound = 1.0
    model.reactions.A_in.upper_bound = 1.0
    
    initial_feasible = check_feasibility_for_all_datapoints_pfba(model, experimental_data)
    print(f"Initial feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
    
    iteration = 0
    while len(initial_feasible) < len(experimental_data) and iteration < max_iterations:
        print(random_reactions_list)
        if not random_reactions_list:
            print("No more random reactions to add.")
            break
        
        # Scoring der Reaktionen
        reaction_scores = {reaction.id: score_reaction_pfba(model, reaction, experimental_data) for reaction in random_reactions_list}
        
        # Auswahl der besten Reaktion basierend auf dem Score
        best_reaction_id = max(reaction_scores, key=reaction_scores.get)
        best_reaction = next(r for r in random_reactions_list if r.id == best_reaction_id)
        
        print(f"Adding reaction: {best_reaction.id} with score: {reaction_scores[best_reaction_id]}")
        model.add_reactions([best_reaction])
        random_reactions_list.remove(best_reaction)
        
        # Prüfen, ob neue Datenpunkte feasible sind
        newly_feasible = check_feasibility_for_all_datapoints_pfba(model, experimental_data)
        newly_feasible_set = set(newly_feasible) - set(initial_feasible)
        
        if newly_feasible_set:
            initial_feasible = newly_feasible
            print(f"Iteration {iteration}: Added {best_reaction.id}. Now feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
            show_flux_distributions_pfba(model, experimental_data)
        else:
            # Entfernen der Reaktion, falls keine neuen Punkte feasible werden
            model.reactions.remove(best_reaction)
            print(f"Iteration {iteration}: Added {best_reaction.id}, but no new feasible data points were found.")
        
        iteration += 1

    print(f"Final feasible data points: {len(initial_feasible)} / {len(experimental_data)}")
    if len(initial_feasible) == len(experimental_data):
        print("All data points are feasible with the added reactions!")
    else:
        print(f"Stopped after {iteration} iterations. Some data points remain infeasible.")

    return model


In [228]:
random.seed(42)
np.random.seed(42)
random_reactions_list = generate_random_reactions(model, 500)

Reaction that is part of the original model is included in this list.

In [229]:
random_reactions_list[19]

Reaction identifier,Random_Rxn_20
Name,Random Reaction 20
Memory address,0x28d2819d0
Stoichiometry,K <=> P <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [230]:
modified_model_pfba_500 = add_best_reaction_pfba(model, experimental_data_zeropadded, random_reactions_list)

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Initial feasible data points: 3 / 11
[<Reaction Random_Rxn_1 at 0x160435310>, <Reaction Random_Rxn_2 at 0x1600a0b50>, <Reaction Random_Rxn_3 at 0x28b0e4050>, <Reaction Random_Rxn_4 at 0x28ac663d0>, <Reaction Random_Rxn_5 at 0x16007bb10>, <Reaction Random_Rxn_6 at 0x16007be50>, <Reaction Random_Rxn_7 at 0x28ac66f50>, <Reaction Random_Rxn_8 at 0x28d280210>, <Reaction Random_Rxn_9 at 0x28d282250>, <Reaction Random_Rxn_10 at 0x28d348cd0>, <Reaction Random_Rxn_11 at 0x28d2828d0>, <Reaction Random_Rxn_12 at 0x28d283350>, <Reaction Random_Rxn_13 at 0x28d283bd0>, <Reaction Random_Rxn_14 at 0x28d2814d0>, <Reaction Random_Rxn_15 at 0x

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Mass balance check doesn't check out
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Mass balance check doesn't check out
Error during optimization: None (infeasible).
Error 

...

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Mass balance check doesn't check out
Error during optimization: None (infeasible).
Mass balance check doesn't check out
Mass balance check doesn't check out
Mass balance check doesn't check out
Mass balance check doesn't check out
Error during optimization: None (infeasible).
Mass balance check doesn't check out
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasibl

In [238]:
modified_model_pfba_500.reactions

[<Reaction A_in at 0x28c968510>,
 <Reaction R1 at 0x28c969550>,
 <Reaction R2 at 0x28c969e50>,
 <Reaction R3 at 0x28c96a110>,
 <Reaction R4 at 0x28c968690>,
 <Reaction R5 at 0x28c96bed0>,
 <Reaction R6 at 0x28c9681d0>,
 <Reaction R7 at 0x28c96b6d0>,
 <Reaction R8 at 0x28c96b010>,
 <Reaction R9 at 0x28c969610>,
 <Reaction R10 at 0x28c96b1d0>,
 <Reaction R11 at 0x28ae3f910>,
 <Reaction R12 at 0x28ae3c2d0>,
 <Reaction E_EX_out at 0x28ae3d750>,
 <Reaction L_EX_out at 0x28ae3e7d0>,
 <Reaction M_EX_out at 0x28ae3cbd0>,
 <Reaction N_EX_out at 0x28ae3ce90>,
 <Reaction P_EX_out at 0x28ae3d550>,
 <Reaction EX_A_EX at 0x28ae3e850>,
 <Reaction EX_E_EX at 0x28ae3d490>,
 <Reaction EX_L_EX at 0x28c97c6d0>,
 <Reaction EX_M_EX at 0x28c97fc90>,
 <Reaction EX_N_EX at 0x28c97ded0>,
 <Reaction EX_P_EX at 0x28c97fb10>,
 <Reaction Random_Rxn_91 at 0x28d321f10>,
 <Reaction Random_Rxn_5 at 0x16007bb10>]

In [240]:
modified_model_pfba_500.reactions.Random_Rxn_91

Reaction identifier,Random_Rxn_91
Name,Random Reaction 91
Memory address,0x28d321f10
Stoichiometry,H + L <=> M + <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [242]:
modified_model_pfba_500.reactions.Random_Rxn_5

Reaction identifier,Random_Rxn_5
Name,Random Reaction 5
Memory address,0x16007bb10
Stoichiometry,H <=> I <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [232]:
reverse_list500 = get_negative_flux_reactions_pfba(modified_model_pfba_500, experimental_data_zeropadded)


Processing Datapoint 1:
Feasible solution found.

Processing Datapoint 2:
Feasible solution found.

Processing Datapoint 3:
Feasible solution found.

Processing Datapoint 4:
Feasible solution found.

Processing Datapoint 5:
Feasible solution found.

Processing Datapoint 6:
Feasible solution found.

Processing Datapoint 7:
Feasible solution found.

Processing Datapoint 8:
Feasible solution found.

Processing Datapoint 9:
Feasible solution found.

Processing Datapoint 10:
Feasible solution found.

Processing Datapoint 11:
Feasible solution found.


In [233]:
print(reverse_list500)

['R11', 'Random_Rxn_91', 'R2', 'R3', 'R10', 'R1', 'EX_A_EX']


In [243]:
random.seed(42)
np.random.seed(42)
random_reactions_list = generate_random_reactions(model, 50)

In [244]:
#Reaction from original model
random_reactions_list[19]

Reaction identifier,Random_Rxn_20
Name,Random Reaction 20
Memory address,0x28c85b890
Stoichiometry,K <=> P <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [245]:
modified_model_pfba_50 = add_best_reaction_pfba(model, experimental_data_zeropadded, random_reactions_list)

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Initial feasible data points: 3 / 11
[<Reaction Random_Rxn_1 at 0x28c97de50>, <Reaction Random_Rxn_2 at 0x28ce96a50>, <Reaction Random_Rxn_3 at 0x1604554d0>, <Reaction Random_Rxn_4 at 0x160454b50>, <Reaction Random_Rxn_5 at 0x28d431e90>, <Reaction Random_Rxn_6 at 0x28d431510>, <Reaction Random_Rxn_7 at 0x28d4326d0>, <Reaction Random_Rxn_8 at 0x28c18a3d0>, <Reaction Random_Rxn_9 at 0x28aed1750>, <Reaction Random_Rxn_10 at 0x28aed0790>, <Reaction Random_Rxn_11 at 0x160454a90>, <Reaction Random_Rxn_12 at 0x160340050>, <Reaction Random_Rxn_13 at 0x160340d50>, <Reaction Random_Rxn_14 at 0x160456550>, <Reaction Random_Rxn_15 at 0x

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (infeasible).
Error during optimization: None (i

In [246]:
modified_model_pfba_50.reactions

[<Reaction A_in at 0x28c3474d0>,
 <Reaction R1 at 0x28c3444d0>,
 <Reaction R2 at 0x28c344450>,
 <Reaction R3 at 0x28c347190>,
 <Reaction R4 at 0x28c345890>,
 <Reaction R5 at 0x28c9ec2d0>,
 <Reaction R6 at 0x28c9ec250>,
 <Reaction R7 at 0x28c9efb50>,
 <Reaction R8 at 0x28c9ec650>,
 <Reaction R9 at 0x28c9efcd0>,
 <Reaction R10 at 0x28c9eead0>,
 <Reaction R11 at 0x28c9ee450>,
 <Reaction R12 at 0x28c9ef710>,
 <Reaction E_EX_out at 0x28c0eaa10>,
 <Reaction L_EX_out at 0x28c0e9bd0>,
 <Reaction M_EX_out at 0x28c0e8f90>,
 <Reaction N_EX_out at 0x28c0e9810>,
 <Reaction P_EX_out at 0x28c0ebcd0>,
 <Reaction EX_A_EX at 0x28b088b10>,
 <Reaction EX_E_EX at 0x28b089610>,
 <Reaction EX_L_EX at 0x28b08aa90>,
 <Reaction EX_M_EX at 0x28b0891d0>,
 <Reaction EX_N_EX at 0x28b08bad0>,
 <Reaction EX_P_EX at 0x28b08a7d0>,
 <Reaction Random_Rxn_5 at 0x28d431e90>,
 <Reaction Random_Rxn_20 at 0x28c85b890>]

In [247]:
modified_model_pfba_50.reactions.Random_Rxn_5

Reaction identifier,Random_Rxn_5
Name,Random Reaction 5
Memory address,0x28d431e90
Stoichiometry,H <=> I <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [248]:
modified_model_pfba_50.reactions.Random_Rxn_20

Reaction identifier,Random_Rxn_20
Name,Random Reaction 20
Memory address,0x28c85b890
Stoichiometry,K <=> P <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Reaction that were used with negative flux for a solution for a datapoint

In [249]:
reverse_list50 = get_negative_flux_reactions_pfba(modified_model_pfba_50, experimental_data_zeropadded)


Processing Datapoint 1:
Feasible solution found.

Processing Datapoint 2:
Feasible solution found.

Processing Datapoint 3:
Feasible solution found.

Processing Datapoint 4:
Feasible solution found.

Processing Datapoint 5:
Feasible solution found.

Processing Datapoint 6:
Feasible solution found.

Processing Datapoint 7:
Feasible solution found.

Processing Datapoint 8:
Feasible solution found.

Processing Datapoint 9:
Feasible solution found.

Processing Datapoint 10:
Feasible solution found.

Processing Datapoint 11:
Feasible solution found.


In [250]:
print(reverse_list50)

['R12', 'Random_Rxn_20', 'Random_Rxn_5', 'EX_A_EX']
